<a href="https://colab.research.google.com/github/kumardesappan/colab-notebooks/blob/main/infer_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://akm-img-a-in.tosshub.com/indiatoday/images/story/201804/jet.jpeg
!wget https://tfhub.dev/tensorflow/lite-model/efficientnet/lite0/fp32/2?lite-format=tflite
!mv /content/2?lite-format=tflite efficientnet-lit0.tflite
!pip3 install tflite_runtime==2.8.0

In [3]:
import math
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite

import time
import os
import numpy as np
import PIL
import PIL.Image as Image

import argparse
model_path ="./efficientnet-lit0.tflite"
input_file ="jet.jpeg"

input_mean =127.5
input_std  =127.5

        

def infer_tflite_model():

  interpreter = tflite.Interpreter(model_path=model_path)
  interpreter.allocate_tensors()
    
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  print(output_details)

  # check the type of the input tensor
  floating_model = input_details[0]['dtype'] == np.float32

  # NxHxWxC, H:1, W:2
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]
  img = Image.open(input_file).resize((width, height))

  # add N dim
  input_data = np.expand_dims(img, axis=0)

  if floating_model:
    input_data = (np.float32(input_data) - input_mean) / input_std

  interpreter.set_tensor(input_details[0]['index'], input_data)

  interpreter.invoke()

  output_data = interpreter.get_tensor(output_details[0]['index'])
  results = np.squeeze(output_data)

  top_k = results.argsort()[-5:][::-1]
  print(top_k)

                                       
infer_tflite_model()



[{'name': 'Softmax', 'index': 163, 'shape': array([   1, 1000], dtype=int32), 'shape_signature': array([   1, 1000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[895 403 744 657  51]
